In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [2]:
import pickle, gzip

traffic_dir = 'traffic_signs_data'

def load_traffic_signs():
    
    with gzip.open(traffic_dir + '/train2.p.gz', mode='rb') as f:
        train = pickle.load(f)
    with gzip.open(traffic_dir + '/test2.p.gz', mode='rb') as f:
        test = pickle.load(f)

    X_train, y_train = train['images'], train['labels']
    X_test, y_test = test['images'], test['labels']
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_traffic_signs()

In [3]:
image_shape = X_train.shape[1:3]
n_classes = np.unique(y_train).shape[0]
'Train:', X_train.shape[0], 'Test:', X_test.shape[0], 'Image shape:', image_shape, 'Classes:', n_classes

('Train:', 39209, 'Test:', 12630, 'Image shape:', (32, 32), 'Classes:', 43)

## Normalize Data

In [4]:
def normalize(image_data, min_x=None, max_x=None):
    min_x = np.min(image_data) if min_x is None else min_x
    max_x = np.max(image_data) if max_x is None else max_x
    a, b = 0.1, 0.9
    result = a + (image_data - min_x) * (b - a) / (max_x - min_x)
    return result, min_x, max_x

In [5]:
X_train_norm, min_x, max_x = normalize(X_train)

In [6]:
X_test_norm, _, _ = normalize(X_test, min_x, max_x)

## LeNet-5

In [7]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    mu = 0.01
    sigma = 0.1
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x, conv1_W, strides = [1, 1, 1, 1], padding='VALID') + conv1_b

    conv1 = tf.nn.relu(conv1)

    # Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean=mu, stddev=sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    conv2 = tf.nn.relu(conv2)

    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Flatten. Input = 5x5x16. Output = 400.
    fc0 = flatten(conv2)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean=mu, stddev=sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1 = tf.matmul(fc0, fc1_W) + fc1_b
    
    fc1 = tf.nn.relu(fc1)

    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W = tf.Variable(tf.truncated_normal(shape=(120, 84), mean=mu, stddev=sigma))
    fc2_b = tf.Variable(tf.zeros(84))
    fc2 = tf.matmul(fc1, fc2_W) + fc2_b
    
    fc2 = tf.nn.relu(fc2)

    # TODO: Layer 5: Fully Connected. Input = 84. Output = 43.
    fc3_W = tf.Variable(tf.truncated_normal(shape=(84, 43), mean=mu, stddev=sigma))
    fc3_b = tf.Variable(tf.zeros(43))
    logits = tf.add(tf.matmul(fc2, fc3_W), fc3_b, name="logits")
    
    return logits

### Train Model

In [8]:
from sklearn.model_selection import train_test_split
X_train_norm, X_valid_norm, y_train, y_valid = train_test_split(X_train_norm, y_train)

In [9]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3), name='x')
y = tf.placeholder(tf.int32, (None), name='y')
one_hot_y = tf.one_hot(y, 43)

In [10]:
rate = 0.001

logits = LeNet(x)
prediction = tf.nn.softmax(logits, name="prediction")
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate, beta1=0.9, beta2=0.999, epsilon=1e-08)
training_operation = optimizer.minimize(loss_operation)

In [11]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

BATCH_SIZE = 64

def evaluate(sess, X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

In [12]:
from sklearn.utils import shuffle

EPOCHS = 15

def train(x_train, y_train, x_valid, y_valid):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        num_examples = len(x_train)

        print("Training...")
        print()
        for i in range(EPOCHS):
            x_train, y_train = shuffle(x_train, y_train)
            for offset in range(0, num_examples, BATCH_SIZE):
                end = offset + BATCH_SIZE
                batch_x, batch_y = x_train[offset:end], y_train[offset:end]
                sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})

            validation_accuracy = evaluate(sess, x_valid, y_valid)

            print("EPOCH {} ...".format(i+1))
            print("Validation Accuracy = {:.3f}".format(validation_accuracy))
            print()

        saver.save(sess, './lenet')
        print("Model saved")

In [13]:
%time train(X_train_norm, y_train, X_valid_norm, y_valid)

Training...

EPOCH 1 ...
Validation Accuracy = 0.724

EPOCH 2 ...
Validation Accuracy = 0.892

EPOCH 3 ...
Validation Accuracy = 0.930

EPOCH 4 ...
Validation Accuracy = 0.941

EPOCH 5 ...
Validation Accuracy = 0.955

EPOCH 6 ...
Validation Accuracy = 0.970

EPOCH 7 ...
Validation Accuracy = 0.960

EPOCH 8 ...
Validation Accuracy = 0.973

EPOCH 9 ...
Validation Accuracy = 0.975

EPOCH 10 ...
Validation Accuracy = 0.975

EPOCH 11 ...
Validation Accuracy = 0.974

EPOCH 12 ...
Validation Accuracy = 0.969

EPOCH 13 ...
Validation Accuracy = 0.977

EPOCH 14 ...
Validation Accuracy = 0.977

EPOCH 15 ...
Validation Accuracy = 0.979

Model saved
CPU times: user 16min 58s, sys: 3min 49s, total: 20min 47s
Wall time: 7min 26s


## Test the model on the test dataset

In [ ]:
with tf.Session() as sess:
    print("Test accuracy = {:.3f}".format(evaluate(sess, X_test_norm, y_test)))

## Try images from the Internet

In [ ]:
import matplotlib.image as mpimg
import cv2

def test_image_picture(file_name, sess):
    img_orig = cv2.cvtColor(cv2.imread(file_name), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img_orig, (32, 32))
    
    pred = sess.run(prediction, feed_dict = { x : [img] })
    index = np.argmax(pred)
    
    top_k_preds = sess.run(tf.nn.top_k(tf.constant(pred), k=3))
    print('Prediction: ', class_names[index])
    print('Top 3 probabilities:', top_k_preds)

    plt.imshow(img_orig)

The sign below is not in our dataset, so we don't count it toward the accuracy.

In [ ]:
test_image_picture('???', s_loaded)